In [1]:
import math
import copy
from tqdm import tqdm

import numpy as np
import pandas as pd

import networkx as nx

import matplotlib.pyplot as plt
import plotly.offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sbemdb import SBEMDB
from cleandb import clean_db, clean_db_uct

from distance import distance
from findpath import find_path

import mapping 

In [2]:
db = SBEMDB()
db = clean_db(db) 
x,y,z = db.segments(444)
mp = mapping.Mapping()

trees Before 398
trees After 46
nodes Before 37481
nodes After 16320
nodecons Before 73994
nodecons After 32548
syncons Before 1199
syncons After 826
synapses Before 552
synapses After 535


In [3]:
node_ids_db = list(db.nodeDetails('tid==444 and typ=6').keys())
segments = pd.read_csv('segments_table.csv')
segments.head()

,segment_id,branch_id,type,depth,is_synapse,node_id,point_node_id
0,0,2137,main,0,False,2137,2137
1,0,2137,main,0,False,3477,2137
2,0,2137,main,0,False,3478,3477
3,0,2137,main,0,False,3479,3478
4,0,2137,main,0,False,3480,3479


In [4]:
'''
Start from beginning node
    find branch point:
        recursively get sum of terminal segments on both ends
        
root_segs = segments[segments['type'] == 'root']['segment_id'].unique() # main, root, intermediate, terminal
types = db.nodetypes()
for seg in root_segs:
    print(seg) #['segment_id']
    
segment_id
branch_id
type
depth
is_synapse
node_id
point_node_id


label branch points: find neighbors of node, only need to look at very first

Node ids are unique..... look at segments[segments['node_id'] == 3485]
'''
node_neighbors = {}
segment_ids = segments['segment_id'].unique()
for i in range(len(segment_ids)):
    sub_data = segments[segments['segment_id'] == segment_ids[i]]
    #first_node_id = sub_data.iloc[-1]['node_id']
    for i in range(len(sub_data)):
        node_id = sub_data.iloc[i]['node_id']
        node_id2 = sub_data.iloc[i]['point_node_id']
        if node_id not in node_neighbors:
            node_neighbors[node_id] = []
        if node_id2 not in node_neighbors:
            node_neighbors[node_id2] = []
        if node_id != node_id2:
            node_neighbors[node_id].append(node_id2)
            node_neighbors[node_id2].append(node_id)
            
print(node_neighbors)

{2137: [3477], 3477: [2137, 3478], 3478: [3477, 3479], 3479: [3478, 3480], 3480: [3479, 3481], 3481: [3480, 3482, 25385], 3482: [3481, 3483, 5462], 3483: [3482, 3484, 5587, 25456], 3484: [3483, 3485], 3485: [3484, 3486, 25467, 25457], 3486: [3485, 4816], 4816: [3486, 4817, 5846], 4817: [4816, 4818, 5589, 6226], 4818: [4817, 4819], 4819: [4818, 4820, 5592, 25554, 17244, 25555], 4820: [4819, 4821], 4821: [4820, 4822], 4822: [4821, 4823, 5730, 6324], 4823: [4822, 4824, 6327, 6339, 25510, 6340], 4824: [4823, 4825], 4825: [4824, 4826], 4826: [4825, 4827], 4827: [4826, 4828, 6377, 6379, 25560], 4828: [4827, 4829], 4829: [4828, 4830], 4830: [4829, 4831], 4831: [4830, 4832], 4832: [4831, 4833], 4833: [4832, 4834, 6515, 4837], 4834: [4833, 4835], 4835: [4834, 4836, 17438], 4836: [4835, 4839], 4839: [4836, 4840, 6551, 25853], 4840: [4839, 4841], 4841: [4840, 4842, 17444, 6556], 4842: [4841, 4843, 25854], 4843: [4842, 4844, 6558, 6665], 4844: [4843, 4845], 4845: [4844, 4846, 6659], 4846: [4845, 4

In [11]:
maxi = 2
item = []
values = list(node_neighbors.values())
for i in values:
    maxi = max(len(i), maxi)
    if len(i) == 7:
        item = i
print(maxi)
item

seven
7


[6561, 6649, 37256, 6563, 25856, 25857, 17449]

In [6]:
'''
3485, 3484
3486, 3485
25467, 3485
25457, 3485
same x2
'''
segments[segments['node_id'] == 3485]

,segment_id,branch_id,type,depth,is_synapse,node_id,point_node_id
9,0,2137,main,0,False,3485,3484
2807,3649,25467,root,1,False,3485,3485
2944,3824,25457,root,1,False,3485,3485


In [7]:
print(segments[segments['node_id'] == 4819])
segments[segments['point_node_id'] == 4819]

     segment_id  branch_id  type  depth  is_synapse  node_id  point_node_id
14            0       2137  main      0       False     4819           4818
183           3       5592  root      1       False     4819           4819


,segment_id,branch_id,type,depth,is_synapse,node_id,point_node_id
15,0,2137,main,0,False,4820,4819
183,3,5592,root,1,False,4819,4819
184,3,5592,root,1,False,5592,4819
4316,7880,25554,ts,1,True,25554,4819
5173,8254,17244,ts,1,False,17244,4819
5616,8392,25555,ts,1,False,25555,4819
